# Introdução

Neste documento faremos os testes de validação do sistema.

In [2]:
# !pip install langchain==0.0.340 langchain --force-reinstall

In [3]:
validation_results_path = './validation_results'

# 1. Obtenção dos Metadados do Minerva

In [4]:
import os
import json
import numpy as np

In [5]:
pdf_folder_path = "./metadata_documents"

## Arquivos de Metadados

In [6]:
filename_list = []
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".json") and not filename.endswith("_x.json"):
        filename_list.append(os.path.join(pdf_folder_path, filename))

filename_list = sorted(filename_list)

In [7]:
filename_list [:5]

['./metadata_documents/documento_.json',
 './metadata_documents/documento_276189.json',
 './metadata_documents/documento_494080.json',
 './metadata_documents/documento_547235.json',
 './metadata_documents/documento_614469.json']

## Obtenção dos Metadados

In [8]:
def get_metadata(filename_list):
    metadata_dict = {}
    
    for filename in filename_list:
        with open(filename, "r") as file: 
            metadata = json.load(file)
            _id = metadata["id"]
            metadata_dict[_id] = metadata
    
    return metadata_dict

In [9]:
metadata_dict = get_metadata(filename_list)

In [10]:
key = list(metadata_dict.keys())[10]

In [11]:
metadata_dict[key]

{'title': 'Mal-estar docente : vozes de professores em uma escola pública de Niterói',
 'authors': ['Santos, Roberta Duarte dos',
  'Carneiro, Cristiana orient.',
  'Universidade Federal do Rio de Janeiro. Instituto de Psicologia.'],
 'abstract': 'A presente pesquisa de Mestrado é dedicada a apresentar algumas considerações a respeito do mal-estar docente sob o viés psicanalítico. Para tanto, utiliza o referencial teórico de Freud e de psicanalistas contemporâneos que vêm se debruçando sobre essa questão. Partindo da escuta realizada em oficinas de rodas de conversas com professores do Instituto de Educação Professor Ismael Coutinho (IEPIC), assim também como de entrevistas individuais com os docentes, foram registradas suas queixas, impotências e impasses. Nesse material, com auxílio da revisão da literatura selecionada, procuramos mapear o mal-estar nas falas dos professores, relacionando-o ao conceito freudiano de mal-estar. A análise das transcrições de tais falas tem nos levado a 

# 2. Seleção dos Testes

In [12]:
import json

In [13]:
validation_tests_partition = '2024-03-10'

validation_tests_path = f'./validation_tests/{validation_tests_partition}'
validation_tests_path

'./validation_tests/2024-03-10'

In [14]:
with open(f'{validation_tests_path}/test_abstract.json', 'r') as fp:
    test_abstract = json.load(fp)

In [15]:
with open(f'{validation_tests_path}/test_keywords.json', 'r') as fp:
    test_keywords = json.load(fp)

In [16]:
with open(f'{validation_tests_path}/test_modified_sentences.json', 'r') as fp:
    test_modified_sentences = json.load(fp)

In [17]:
with open(f'{validation_tests_path}/test_sentences.json', 'r') as fp:
    test_sentences = json.load(fp)

In [18]:
with open(f'{validation_tests_path}/test_title.json', 'r') as fp:
    test_title = json.load(fp)

# 3. Vector Search

In [19]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

from lib.retriever import Retriever
from lib.reranker import Reranker

In [20]:
print("Recuperando Embeddings...")
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

Recuperando Embeddings...


In [21]:
print("Construindo Vector Database...")
vectorstore = FAISS.load_local("./faiss_vector_store/", embeddings)

Construindo Vector Database...


In [22]:
openai_api_key = "xxxxxxxxxxxx"
cohere_key = "xxxxxxxxxx"

In [23]:
retriever = Retriever(vectorstore)
retriever1 = Retriever(vectorstore, MINIMUM_RESULTS_PER_DOCUMENT=1)
reranker = Reranker(cohere_key=cohere_key, model="rerank-multilingual-v2.0")

# 4. LLM

In [24]:
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI

In [25]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature="0.5",
    openai_api_key=openai_api_key,       
    max_tokens=2000
)

# Teste 1. Recuperação Vector Search por Abstracts

Neste teste ainda temos alguns pontos abertos, como os parametros de k para recuperação.

*Cohere Embedding Limitations:*<br>
The total max chunks (length of documents * max_chunks_per_doc) must be less than 10000.
We recommend a maximum of 1,000 documents for optimal endpoint performance.

In [26]:
import time

In [60]:
def get_index_of_element(list_scores, _id):
    count = 0
    for score in list_scores:
        if score['id'] == _id:
            return count
        count += 1
    return -1

In [70]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [100]:
similar(
    "A religiosidade e a espiritualidade (R/E) têm sido foco de grande interesse de pesquisadores, pois se mostram presentes na vida de muitos indivíduos, representando formas de vivenciar o sagrado e o transcendente. Sendo assim, é possível que o tema surja com frequência no contexto de psicoterapia e que influencie vários aspectos da vida dos pacientes.",
    "A religiosidade e a espiritualidade (R/E) têm sido foco de grande interesse de pesquisadores, pois se mostram presentes na vida de muitos indivíduos, representando formas de vivenciar o sagrado e o transcendente. Sendo assim, é possível que o tema surja com frequência no contexto de psicoterapia e que influencie vários aspectos da vida dos pacientes."
)

1.0

In [106]:
similar(
    "A religiosidade e a espiritualidade (R/E) têm sido foco de grande interesse de pesquisadores, pois se mostram presentes na vida de muitos indivíduos, representando formas de vivenciar o sagrado e o transcendente. Sendo assim, é possível que o tema surja com frequência no contexto de psicoterapia e que influencie vários aspectos da vida dos pacientes.",
    "A religioxsidade e aespirxxxitualidade (R/E) têm sido foco de grandeintexxxresse de pesquisa ores, pois se mxxx 123 ostram presentes na vida de muitos indivíd uos, representando formas de vivenciar o sagradoe o transcendente. Sendo assim, é possível que oema surja com frequência no contexto de psico terapia e que ncie vá  rios aspec tos da vida dos pacientes."
)

0.8723702664796634

In [101]:
similar(
    "A religiosidade e a espiritualidade (R/E) têm sido foco de grande interesse de pesquisadores, pois se mostram presentes na vida de muitos indivíduos, representando formas de vivenciar o sagrado e o transcendente. Sendo assim, é possível que o tema surja com frequência no contexto de psicoterapia e que influencie vários aspectos da vida dos pacientes.",
    "A religiosidade e a têm sido foco de grande interesse de pesquisadores, pois se mostram vida de muitos indivíduos, de vivenciar o sagrado e o transcendente. é possível que o contexto de psicoterapia e que influencie vários aspectos da vida dos."
)

0.3187919463087248

In [96]:
similar(
    "A religiosidade e a espiritualidade (R/E) têm sido foco de grande interesse de pesquisadores, pois se mostram presentes na vida de muitos indivíduos, representando formas de vivenciar o sagrado e o transcendente. Sendo assim, é possível que o tema surja com frequência no contexto de psicoterapia e que influencie vários aspectos da vida dos pacientes.",
    "A religiosidade e a espiritualidade (R/E) têm sido foco de grande interesse de pesquisadores, pois se mostram presentes na vida de muitos indivíduos, representando formas de vivenciar o sagrado e o transcendente."
)

0.75177304964539

In [98]:
similar(
    "A religiosidade e a espiritualidade (R/E) têm sido foco de grande interesse de pesquisadores, pois se mostram presentes na vida de muitos indivíduos, representando formas de vivenciar o sagrado e o transcendente. Sendo assim, é possível que o tema surja com frequência no contexto de psicoterapia e que influencie vários aspectos da vida dos pacientes.",
    "representando formas de vivenciar o sagrado e o transcendente. Sendo assim, é possível que o tema surja com frequência no contexto de psicoterapia e que influencie vários aspectos da vida dos pacientes."
)

0.7292418772563177

In [99]:
similar(
    "A religiosidade e a espiritualidade (R/E) têm sido foco de grande interesse de pesquisadores, pois se mostram presentes na vida de muitos indivíduos, representando formas de vivenciar o sagrado e o transcendente. Sendo assim, é possível que o tema surja com frequência no contexto de psicoterapia e que influencie vários aspectos da vida dos pacientes.",
    "No entanto, o assunto pode não ser contemplado por psicoterapeutas na sua prática clínica. Isso pode ocorrer pela ideia de que não seria possível abordar o assunto no contexto da psicoterapia baseada em evidências científicas, como é o caso da Terapia Cognitivo-Comportamental."
)

0.04769475357710652

In [ ]:
for abstract_obj in test_abstract:
    print(">", abstract_obj['id'])

    abstract = abstract_obj['abstract']
    doc_id = abstract_obj['id']

    retrieved_documents = await retriever.search(
        abstract, k=1000, fetch_k=100000, max_k_by_document=5, filter={"type":"sentences"}
    )

#     print(abstract[:600])

#     print("")

    docs_mean = []
    docs_median = []
    docs_min = []
    for idx, docs in retrieved_documents.items():
        scores = [doc[0] for doc in docs if doc[0] > 0.15 and similar(abstract, doc[1].page_content) < 0.7] # removes sentence == abstract
        texts = [doc[1].page_content for doc in docs if doc[0] > 0.15 and similar(abstract, doc[1].page_content) < 0.7]

        mean = np.mean(scores)
        median = np.median(scores)
        minimum = np.inf if len(scores) == 0 else np.min(scores)
        # print(idx, "len", len(scores), mean, median, minimum, "scores:", scores)

        # print(texts[0][:600])
        # print(similar(abstract, texts[0]))

        docs_mean.append({"id": idx, "score": mean})
        docs_median.append({"id": idx, "score": median})
        docs_min.append({"id": idx, "score": minimum})

        # print("")

    docs_mean = sorted(docs_mean, key=lambda elem: elem['score'])
    docs_median = sorted(docs_median, key=lambda elem: elem['score'])
    docs_min = sorted(docs_min, key=lambda elem: elem['score'])

    print(get_index_of_element(docs_mean, doc_id))
    print(get_index_of_element(docs_median, doc_id))
    print(get_index_of_element(docs_min, doc_id))

> 276189
0
0
0
> 494080
0
0
0
> 547235
0
0
0
> 614469
0
0
0
> 683353
-1
-1
-1
> 762801
0
0
0
> 768599
1
1
1
> 774811
-1
-1
-1
> 806362
13
14
4
> 814014
0
0
0
> 81683
-1
-1
-1
> 829660
0
0
0
> 842988
-1
-1
-1
> 856777
0
0
0
> 861483
0
0
0
> 868374
1
1
1
> 873196
0
0
0
> 880394
0
0
0
> 881375
0
0
0
> 883182
0
0
0
> 883218
0
0
0
> 883377
1
1
0
> 883397
0
0
0
> 883402
2
1
2
> 883419
0
0
0
> 883546
0
0
0
> 883556
0
0
0
> 883689
0
0
0
> 883720
19
29
6
> 884143
0
0
0
> 884145
0
0
2
> 884150
5
5
12
> 884154
0
0
0
> 884425
0
0
0
> 884532
0
0
0
> 884536
-1
-1
-1
> 884641
8
9
10
> 884656
0
0
0
> 884744
0
0
0
> 884752
0
0
0
> 885010


In [54]:
test_1_execution = []

for abstract_obj in test_abstract[:50]:
    abstract = abstract_obj['abstract']
    doc_id = abstract_obj['id']
    
    retrieved_documents = await retriever.search(
        abstract, k=1000, fetch_k=100000, max_k_by_document=5, filter={"type":"sentences"}
    )

    documents = reranker.prepare_documents(metadata_dict, retrieved_documents)
    reranked_documents = reranker.rerank(abstract, documents, 10, using=["sentences"])
    # print(documents)

    ordered_docs = [{"id": doc["id"], "relevance_score": doc["relevance_score"]} for doc in reranked_documents]
    test_1_execution.append({"id": doc_id, "result": ordered_docs})
    
    time.sleep(10)

Query has been truncated from the right to 256 tokens from 939 tokens.
Query has been truncated from the right to 256 tokens from 605 tokens.
Query has been truncated from the right to 256 tokens from 318 tokens.
Query has been truncated from the right to 256 tokens from 1531 tokens.
Query has been truncated from the right to 256 tokens from 345 tokens.
Query has been truncated from the right to 256 tokens from 658 tokens.
Query has been truncated from the right to 256 tokens from 271 tokens.
Query has been truncated from the right to 256 tokens from 411 tokens.
Query has been truncated from the right to 256 tokens from 537 tokens.
Query has been truncated from the right to 256 tokens from 303 tokens.
Query has been truncated from the right to 256 tokens from 535 tokens.
Query has been truncated from the right to 256 tokens from 804 tokens.
Query has been truncated from the right to 256 tokens from 282 tokens.
Query has been truncated from the right to 256 tokens from 859 tokens.
Query

In [55]:
# Salvamento do resultado

# with open(f'{validation_results_path}/test_1_execution.json', 'w') as fp:
#     json.dump(test_1_execution, fp)

In [56]:
test_1_scores = []
for exec in test_1_execution:
    score = None
    for res in exec['result']:
        if res['id'] == exec['id']:
            score = res['relevance_score']
    
    test_1_scores.append(score)

In [57]:
test_1_scores

[0.99999934,
 0.9998013,
 0.9952448,
 0.23846151,
 None,
 0.99999934,
 0.99998105,
 None,
 0.99993545,
 0.99998707,
 None,
 0.9999996,
 None,
 0.9999985,
 0.99999976,
 0.9999685,
 0.9999998,
 0.9999998,
 0.9999981,
 0.999991,
 0.9998823,
 0.9815951,
 0.9994807,
 0.99993134,
 0.9999809,
 0.999996,
 0.9999996,
 0.9999829,
 0.9995505,
 0.99999887,
 0.9998676,
 0.9999755,
 0.998835,
 0.9996513,
 0.9999998,
 None,
 None,
 0.99999934,
 0.9999994,
 0.8949732,
 0.99999976,
 0.99999696,
 0.99718815,
 0.99999696,
 0.99999636,
 0.99999905,
 0.999935,
 0.99985117,
 0.9999528,
 0.9999987]

# Teste 2: Recuperação Vector Search por Abstracts Parafraseados

In [24]:
import time

In [40]:
abstract_obj = test_abstract[12]
print(">", abstract_obj['id'])

abstract = abstract_obj['abstract']
doc_id = abstract_obj['id']

query = f"""
A seguir, está um resumo de um artigo científico:

{abstract}

Retorne um resumo com no máximo 100 palavras e com escrita simples, em português:
"""

messages = [SystemMessage(content=query)]
abstract = llm(messages).content

print(abstract, "\n")

retrieved_documents = await retriever.search(
    abstract, k=1000, fetch_k=100000, max_k_by_document=5, filter={"type":"sentences"}
)

count = 0
for idx, docs in retrieved_documents.items():
    scores = [doc[0] for doc in docs]
        
    print(idx, np.mean(scores), np.median(scores), "scores:", scores)
    if idx == abstract_obj['id']:
        print(">", count, np.mean(scores), np.median(scores))
    
    count += 1

> 842988
O artigo aborda o desenvolvimento de métodos computacionais para estudar a interação de átomos e moléculas com radiação em diferentes faixas de frequência, desde o ultravioleta até raios-X. Foram utilizadas técnicas como TDDFT e CC para analisar a fotoabsorção e fotoionização de diversas substâncias, como benzeno e água, em estados fundamentais e excitados. Os resultados mostram a eficácia dessas abordagens na obtenção de informações sobre as propriedades eletrônicas e espectrais desses sistemas, incluindo o papel das ligações de hidrogênio em moléculas como o ácido fórmico. 

920848 1.7540516 1.7569684 scores: [1.5443864, 1.7113216, 1.7569684, 1.8723807, 1.885201]
911280 1.7790353 1.8756056 scores: [1.5772862, 1.6793933, 1.8756056, 1.8776317, 1.8852603]
918355 2.0631623 2.145453 scores: [1.6467042, 1.7508944, 2.145453, 2.3389368, 2.4338236]
940981 1.9041357 2.0000498 scores: [1.6921196, 1.7786847, 2.0000498, 2.013498, 2.0363266]
933868 2.1462436 2.2285793 scores: [1.7760398, 

In [25]:
test_2_execution = []

for abstract_obj in test_abstract[:50]:
    abstract = abstract_obj['abstract']
    doc_id = abstract_obj['id']
    
    query = f"""
    A seguir, está um resumo de um artigo científico:
    
    {abstract}
    
    Retorne um resumo com no máximo 100 palavras e com escrita simples, em português:
    """
    
    messages = [SystemMessage(content=query)]
    abstract = llm(messages).content
    print(abstract, "\n")
    
    retrieved_documents = await retriever.search(
        abstract, k=500, fetch_k=100000, max_k_by_document=5, filter={"type":"sentences"}
    )

    documents = reranker.prepare_documents(metadata_dict, retrieved_documents)
    
    # IMPORTANTE: aqui eu uso apenas as sentences no documento de rerank, para evitar contaminacao
    reranked_documents = reranker.rerank(abstract, documents, 10, using=["sentences"])

    ordered_docs = [{"id": doc["id"], "relevance_score": doc["relevance_score"]} for doc in reranked_documents]
    test_2_execution.append({"id": doc_id, "result": ordered_docs})
    
    time.sleep(10)

O estudo analisa o trabalho de psicólogos no NASF do Rio de Janeiro, destacando desafios como sobrecarga, precarização e falta de profissionais. Identifica estratégias de prazer e defesa dos trabalhadores, ressaltando a importância da formação. O coletivo é visto como essencial para lidar com as precarizações na saúde. O estudo sugere ampliação para outras regiões e destaca a necessidade de investigações atualizadas diante das mudanças nos vínculos empregatícios e desafios em saúde na cidade. 

O artigo investigou como terapeutas cognitivo-comportamentais lidam com a religiosidade e espiritualidade de pacientes no Brasil. A maioria dos terapeutas se considerava religiosa e espiritualizada, sendo que a abordagem do tema estava mais relacionada à crença pessoal do terapeuta do que à sua formação profissional. Os resultados destacaram a importância de capacitar psicólogos para discutir R/E na terapia. Recomenda-se mais iniciativas educacionais e pesquisas sobre o assunto, incluindo estudo

*Observações:<br>
Este teste com k=500 está usando 3 unidades de reranking do cohere por teste*

In [26]:
test_2_scores = []
for exec in test_2_execution:
    score = None
    for res in exec['result']:
        if res['id'] == exec['id']:
            score = res['relevance_score']
    
    test_2_scores.append(score)

In [27]:
test_2_scores

[0.9998523,
 0.99998844,
 0.98875546,
 0.99589807,
 None,
 0.99999934,
 0.99999243,
 None,
 0.99999815,
 0.9952633,
 None,
 0.99999905,
 None,
 0.9999947,
 0.99988765,
 0.73885757,
 0.9999994,
 0.99999976,
 0.89606965,
 0.9999987,
 0.9999779,
 0.9904775,
 0.99966204,
 0.9999165,
 0.99999607,
 0.99999845,
 0.9956347,
 0.9996486,
 0.9999971,
 0.9999553,
 0.9997063,
 0.99995667,
 0.9996673,
 0.99999803,
 0.9999992,
 None,
 0.99899167,
 0.999995,
 0.9678993,
 0.8493475,
 0.9999996,
 0.99984765,
 0.9988531,
 0.9997508,
 0.9984084,
 0.9996513,
 0.99696034,
 0.9999061,
 0.9189179,
 0.9613165]

# Teste 3: Recuperação Vector Search por Abstracts tirando seus Objetivos

In [26]:
import time

In [27]:
test_3_execution = []

for abstract_obj in test_abstract[:50]:
    abstract = abstract_obj['abstract']
    doc_id = abstract_obj['id']
    
    query = f"""
    A seguir, está um resumo de um artigo científico:
    
    {abstract}
    
    Retorne, com escrita simples, quais os objetivos da pesquisa, em português e com no máximo 200 palavras:
    """
    
    messages = [SystemMessage(content=query)]
    abstract = llm(messages).content
    print(doc_id, ":", abstract, "\n")
    
    retrieved_documents = await retriever.search(
        abstract, k=500, fetch_k=100000, max_k_by_document=5, filter={"type":"sentences"}
    )

    documents = reranker.prepare_documents(metadata_dict, retrieved_documents)
    
    # IMPORTANTE: aqui eu uso apenas as sentences no documento de rerank, para evitar contaminacao
    reranked_documents = reranker.rerank(abstract, documents, 10, using=["sentences"])

    ordered_docs = [{"id": doc["id"], "relevance_score": doc["relevance_score"]} for doc in reranked_documents]
    test_3_execution.append({"id": doc_id, "result": ordered_docs})
    
    time.sleep(7)

276189 : O objetivo da pesquisa foi analisar a organização do trabalho dos psicólogos que atuam nas equipes do Núcleo de Apoio à Saúde da Família (NASF) no município do Rio de Janeiro, especificamente na área programática 2.1. Para isso, o estudo buscou compreender o contexto de trabalho atual desses profissionais na atenção básica, identificar o trabalho prescrito e o realizado, investigar as vivências de sofrimento e prazer dos psicólogos em seu ambiente de trabalho, além de analisar as estratégias utilizadas para efetivar a prática profissional. A pesquisa foi realizada por meio de entrevistas com psicólogos que integram equipes NASF da Zona Sul do município, entre os anos de 2019 e 2020, utilizando a Análise dos Núcleos de Sentido como metodologia de análise dos dados. Os resultados obtidos visam contribuir para a compreensão da realidade desses profissionais e apontar possíveis melhorias nas condições de trabalho e na efetividade dos serviços prestados. 



Query has been truncated from the right to 256 tokens from 267 tokens.


494080 : O objetivo da pesquisa foi investigar como os terapeutas cognitivo-comportamentais lidam com a religiosidade e espiritualidade (R/E) de seus pacientes no contexto da psicoterapia. Para isso, foram aplicados questionários em 263 terapeutas que atuam no Brasil, buscando verificar suas percepções e opiniões em relação à R/E dos pacientes. Os resultados obtidos demonstraram que a maioria dos terapeutas se considerava religiosa, espiritualizada ou ambos, e que a abordagem da R/E no contexto terapêutico estava mais relacionada com a própria religiosidade, espiritualidade ou ausência destas do terapeuta do que com sua capacitação profissional. Diante disso, os pesquisadores sugerem a necessidade de iniciativas educacionais e formativas para capacitar os terapeutas cognitivo-comportamentais e psicólogos de modo a lidar de forma mais adequada com a R/E dos pacientes, visando aprofundar o conhecimento sobre o tema e ampliar a compreensão em diferentes contextos religiosos e espirituais.

Query has been truncated from the right to 256 tokens from 275 tokens.


547235 : O objetivo da pesquisa foi analisar os romances de Tahar Ben Jelloun, L'Enfant de sable, La nuit sacrée e La Nuit de l'erreur, sob a perspectiva da teatralidade do narrador-contador. Para isso, foi necessário investigar a presença dessa teatralidade e do personagem do narrador-contador nos romances, utilizando como base a teoria do texto e a intertextualidade. A pesquisa buscou compreender as bifurcações presentes nas narrativas, que se fragmentam em diferentes olhares, vozes, contadores e versões. Além disso, o estudo visou destacar o papel do escritor como testemunha e mediador, que reterritorializa culturas e tradições magrebinas. Por fim, a pesquisa explorou a praça como cenário da teatralidade do contador, que transita entre culturas, tradições, passado e presente, utilizando estratégias metanarrativas para agenciar a pluralidade dialógica e narrativa nas obras analisadas. 



...

885153 : O objetivo da pesquisa é analisar a obra do escritor marroquino Abdellah Taïa, que publica romances e contos em língua francesa desde 2004. Através de uma abordagem autoficcional e biográfica, o estudo visa investigar como suas ficções exploram identidades deslocadas em espaços nativos e diaspóricos, transitando por culturas distintas e desafiando normas sexuais e religiosas. O foco recai sobre a materialidade corpórea dos sujeitos narrativos dissidentes, que se apropriam de enunciados e línguas alheias para representar a si mesmos, em uma postura de resistência ao colonialismo e nacionalismo dominantes. A pesquisa busca compreender como esses corpos, mesclando referências djinn e queer, revisam e desconstroem imaginários identitários historicamente construídos, confrontando agentes detentores dos meios expressivos da fala e questionando relações de dominação e violência em contextos nativos e europeus. 



Query has been truncated from the right to 256 tokens from 259 tokens.


885376 : O objetivo da pesquisa foi analisar a cadeia produtiva de fertilizantes organominerais no Brasil, desde a obtenção de insumos até a comercialização do produto final, com foco nos processos de registro regulamentados pelo Ministério da Agricultura, Pecuária e Abastecimento (MAPA). A pesquisa buscou mapear a logística, matérias-primas, quantidades produzidas e empresas produtoras, além de desenvolver um sistema de gerenciamento de banco de dados para unificar e organizar as informações coletadas. Também foi objetivo do estudo analisar a produção de patentes e publicações científicas sobre fertilizantes organominerais, identificando os países mais ativos nesse sentido. Com isso, a pesquisa buscou fornecer subsídios para a melhoria da organização e controle de informações no setor, evidenciando o desenvolvimento e crescimento da indústria de fertilizantes organominerais e a necessidade de aprimoramentos contínuos por parte dos órgãos responsáveis. 



Query has been truncated from the right to 256 tokens from 263 tokens.


885459 : O objetivo da pesquisa foi realizar uma investigação detalhada sobre a produção açucareira na Baixada de Jacarepaguá, no Rio de Janeiro colonial, com foco nos engenhos que ali se instalaram a partir do século XVI. Através de estudos arqueológicos, especialmente no Engenho do Camorim, buscou-se compreender a implantação dos engenhos na paisagem, analisar as estruturas arquitetônicas, os espaços de trabalho e a cultura material relacionada a esses complexos açucareiros. Além disso, a pesquisa teve como objetivo acessar a materialidade dos diferentes grupos étnicos que foram utilizados como mão-de-obra nos engenhos. Dessa forma, o estudo visou preencher lacunas na arqueologia do Rio de Janeiro, especialmente no que diz respeito aos engenhos açucareiros, contribuindo para uma melhor compreensão do passado histórico e cultural da região. 

885644 : O objetivo da pesquisa é apresentar novas informações e discussões sobre a escravidão, focando na paisagem do Valongo, no Rio de Janeir

In [29]:
test_3_scores = []
for exec in test_3_execution:
    score = None
    for res in exec['result']:
        if res['id'] == exec['id']:
            score = res['relevance_score']
    
    test_3_scores.append(score)

In [30]:
test_3_scores

[0.9999977,
 0.9999907,
 0.85717136,
 0.99916714,
 None,
 0.9999996,
 0.99998766,
 None,
 0.99965936,
 0.9999685,
 None,
 0.99999934,
 None,
 0.99999887,
 0.99998987,
 None,
 0.99999976,
 0.9999996,
 0.9995661,
 0.99998236,
 0.9999981,
 0.97741395,
 0.9999761,
 0.9999234,
 0.99999243,
 0.9986489,
 0.9421884,
 0.9999876,
 0.9999709,
 0.9999673,
 0.9987744,
 0.9999974,
 0.9999794,
 0.9999635,
 0.9999928,
 None,
 None,
 0.999989,
 0.9997766,
 0.984516,
 0.9999968,
 0.999924,
 0.9999907,
 0.99767685,
 0.9999593,
 0.99999774,
 0.99968,
 0.9998313,
 0.99998766,
 0.99994856]

# Teste 4. Recuperação Vector Search por Keywords

In [31]:
import time

In [32]:
test_4_execution = []

for keywords_obj in test_keywords[:50]:
    keywords = keywords_obj['keywords']
    doc_id = keywords_obj['id']
    
    retrieved_documents = await retriever.search(
        keywords, k=500, fetch_k=100000, max_k_by_document=5, filter={"type":"sentences"}
    )

    documents = reranker.prepare_documents(metadata_dict, retrieved_documents)
    reranked_documents = reranker.rerank(keywords, documents, 10, using=["sentences"])
    # print(documents)

    ordered_docs = [{"id": doc["id"], "relevance_score": doc["relevance_score"]} for doc in reranked_documents]
    test_4_execution.append({"id": doc_id, "result": ordered_docs})
    
    time.sleep(10)

In [ ]:
# Salvamento do resultado

# with open(f'{validation_results_path}/test_1_execution.json', 'w') as fp:
#     json.dump(test_1_execution, fp)

In [33]:
test_4_scores = []
for exec in test_4_execution:
    score = None
    for res in exec['result']:
        if res['id'] == exec['id']:
            score = res['relevance_score']
    
    test_4_scores.append(score)

In [34]:
test_4_scores

[0.9612438,
 0.9974191,
 None,
 0.9249593,
 None,
 0.7036513,
 None,
 None,
 0.12689333,
 0.6931615,
 None,
 0.92630374,
 None,
 0.004736749,
 0.7657686,
 0.9900251,
 0.999547,
 None,
 0.6490227,
 0.74927074,
 None,
 0.92007446,
 0.81537956,
 0.9996724,
 0.99887085,
 None,
 0.80639565,
 0.9986594,
 0.9897117,
 0.99215376,
 0.99999547,
 0.9999996,
 0.86281466,
 None,
 None,
 0.025371311,
 None,
 0.9583042,
 0.8577681,
 0.6286354,
 0.28427786,
 0.51198345,
 0.9786719,
 0.9248236,
 0.93257624,
 0.011419615,
 0.9999587,
 0.9980506,
 0.9983959,
 None]

# Teste 5. Recuperação Vector Search por Sentences

In [35]:
import time

In [36]:
test_5_execution = []

for sentences_obj in test_sentences[:50]:
    sentences = sentences_obj['sentence']
    doc_id = sentences_obj['id']
    
    retrieved_documents = await retriever1.search(
        sentences, k=500, fetch_k=100000, max_k_by_document=5, filter={"type":"abstract"}
    )

    documents = reranker.prepare_documents(metadata_dict, retrieved_documents)
    reranked_documents = reranker.rerank(sentences, documents, 10, using=["abstract"])
    # print(documents)

    ordered_docs = [{"id": doc["id"], "relevance_score": doc["relevance_score"]} for doc in reranked_documents]
    test_5_execution.append({"id": doc_id, "result": ordered_docs})
    
    time.sleep(10)

In [37]:
# Salvamento do resultado

# with open(f'{validation_results_path}/test_1_execution.json', 'w') as fp:
#     json.dump(test_1_execution, fp)

In [38]:
# test_5_execution

In [39]:
test_5_scores = []
for exec in test_5_execution:
    score = None
    for res in exec['result']:
        if res['id'] == exec['id']:
            score = res['relevance_score']
    
    test_5_scores.append(score)

In [40]:
test_5_scores

[None,
 None,
 None,
 0.09790669,
 None,
 0.014559519,
 0.004663655,
 None,
 0.04401865,
 None,
 0.03004337,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.9998546,
 0.31965208,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.8586001,
 0.0107351625,
 0.16000335,
 None,
 0.017045317,
 0.10989668,
 0.9437637,
 None,
 0.36579645,
 None,
 None,
 0.018159512,
 None,
 0.0004373344,
 None]

# Teste 6. Recuperação Vector Search por Title

In [41]:
import time

In [42]:
test_6_execution = []

for title_obj in test_title[:50]:
    title = title_obj['title']
    doc_id = title_obj['id']
    
    retrieved_documents = await retriever.search(
        title, k=500, fetch_k=100000, max_k_by_document=5, filter={"type":"sentences"}
    )

    documents = reranker.prepare_documents(metadata_dict, retrieved_documents)
    reranked_documents = reranker.rerank(title, documents, 10, using=["sentences"])
    # print(documents)

    ordered_docs = [{"id": doc["id"], "relevance_score": doc["relevance_score"]} for doc in reranked_documents]
    test_6_execution.append({"id": doc_id, "result": ordered_docs})
    
    time.sleep(10)

In [43]:
# Salvamento do resultado

# with open(f'{validation_results_path}/test_1_execution.json', 'w') as fp:
#     json.dump(test_1_execution, fp)

In [44]:
# test_5_execution

In [45]:
test_6_scores = []
for exec in test_6_execution:
    score = None
    for res in exec['result']:
        if res['id'] == exec['id']:
            score = res['relevance_score']
    
    test_6_scores.append(score)

In [46]:
test_6_scores

[0.9999729,
 0.999764,
 0.9999542,
 0.2670257,
 None,
 0.99883956,
 0.999844,
 None,
 0.9999999,
 0.9999998,
 None,
 0.9999777,
 0.9999431,
 0.9999999,
 0.99990463,
 0.9999999,
 0.9999999,
 0.9999998,
 0.9999999,
 0.9999999,
 0.9999994,
 0.8724561,
 0.9999999,
 0.9999997,
 0.9910493,
 0.4642032,
 0.9999999,
 0.9999964,
 0.9999999,
 0.99999976,
 0.9999999,
 0.9999999,
 0.99953264,
 0.9999999,
 None,
 0.9999999,
 0.95791227,
 0.9999991,
 0.9999999,
 0.9999998,
 0.99999934,
 0.9999794,
 0.9999883,
 None,
 0.99998605,
 0.999994,
 0.9999999,
 0.9999999,
 0.9999875,
 0.8969756]